In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py
import numpy as np
from scipy import constants, signal, stats, optimize
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cycler import cycler

import analysis
from basic.paths import (
    RESULTS_FOLDER,
    PARTICLE_VARIATION_FOLDER,
    DENSITY_VARIATION_FOLDER,
    V_FLOW_VARIATION_FOLDER,
    FOLDER_1D,
    FOLDER_2D,
    MPLSTYLE_FILE
)
from basic import (
    physics,
    Species,
    SpeciesInfo,
    RunInfo,
    Distribution
)

from plots import (
    settings,
    plots_1D,
    plots_2D,
    general,
)

info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

save=True
plt.style.use(MPLSTYLE_FILE)
matplotlib.rcParams['figure.dpi'] = 100

In [ ]:
# visualization of distribution overlap with interaction region
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def interactive_plot(u_alpha):
    width = 40
    c_s = info.c_s * 1e-3
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)

    t = np.linspace(0, c_s)
    s = np.linspace(-5 * c_s, 5 * c_s)
    plt.figure()
    # plt.text(0, 0, f"{u_alpha} {theta}")
    circle = plt.Circle((0,0), radius=c_s, fill=False, ls="--", lw=2)
    plt.plot(np.cos(theta) * t, np.sin(theta) * t, ls="solid")
    plt.plot(np.cos(theta) * t, -np.sin(theta) * t, ls="solid")
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, c_s * np.sin(theta) - np.cos(theta) * s)
    plt.plot(c_s * np.cos(theta) + np.sin(theta) * s, -c_s * np.sin(theta) + np.cos(theta) * s)
    rect_pos = plt.Rectangle(
        xy=(
            (c_s - width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            (c_s - width / 2) * np.sin(theta) - np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=theta * 180 / np.pi+270, alpha=0.4, edgecolor="black"
    )
    rect_neg = plt.Rectangle(
        xy=(
            +(c_s + width / 2) * np.cos(theta) + np.sin(theta) * (-2 * c_s),
            -(c_s + width / 2) * np.sin(theta) + np.cos(theta) * (-2 * c_s)
        ),
        width=1000, height=width, angle=-theta * 180 / np.pi+90, alpha=0.4, edgecolor="black"
    )
    plt.scatter(u_alpha, 0, s=40,zorder=5)
    plt.gca().add_patch(circle)
    plt.gca().add_patch(rect_pos)
    plt.gca().add_patch(rect_neg)
    plt.gca().set_aspect("equal")
    plt.xlim(-110, u_alpha+width)
    plt.ylim(-110, 110)
    plt.xlabel("Velocity $v_{\\alpha,x}$ (km/s)")
    plt.ylabel("Velocity $v_{\\alpha,y}$ (km/s)")
    plt.show()

interactive_plt = interactive(interactive_plot, u_alpha=(100.0, 200.0))
interactive_plt

In [ ]:
magnetic_field_strength = 4.346348316425957e-06# 45e-9
omega_c_alpha = magnetic_field_strength * info.alpha.si_charge / info.alpha.si_mass
omega_c_proton = magnetic_field_strength * info.proton.si_charge / info.proton.si_mass
print(f"B = {magnetic_field_strength * 1e4}")
print(info.omega_pp / omega_c_alpha)
print(info.omega_pp / omega_c_proton)
print(info.omega_pp / (10 * info.proton.si_charge / info.proton.si_mass))

In [ ]:
magnetic_field_strength / np.sqrt(2)

In [ ]:
1 / (magnetic_field_strength / np.sqrt(2)) ** 2 /(1 / (magnetic_field_strength) ** 2 )

In [ ]:
from scipy import special, integrate
def distr_integral(u_alpha, w, m, T):
    c_s = info.c_s
    k = constants.k
    if u_alpha < c_s:
        theta = 0
    else:
        theta = np.arccos(c_s / u_alpha)
    if theta < 1e-15:
        integral_band = special.erf(w * np.sqrt(m / (8 * k * T)))
        return integral_band
    f_geom = w / np.sin(theta)
    e_geom = w / np.cos(theta)

    integral_band = special.erf(w * np.sqrt(m / (8 * k * T)))
    integral_triangle, _ = integrate.quad(
        func=lambda v_x: np.exp(- m * v_x ** 2 / (2 * k * T)) * \
        special.erf((f_geom * (e_geom - 2 * v_x)) / (np.sqrt(8 * k * T / m) * e_geom)) / \
        np.sqrt(8 * np.pi * k * T / m),
        a=0, b=e_geom / 2
    )
    return 2 * integral_band - 4 * integral_triangle
wave_length = 0.5 * 2 * np.pi * info.lambda_D / 0.7
interaction_width = np.sqrt(
    2 * constants.elementary_charge * 0.8 * wave_length / info.proton.si_mass
)
x = np.linspace(1e5, 2e5, num=1000)
y = np.array([
    distr_integral(u_alpha, w=interaction_width, m=info.alpha.si_mass, T=info.alpha.si_temperature) for u_alpha in x
])
plt.plot(x * 1e-3, y)
plt.xlabel("Initial flow velocity $u_\\alpha$ (km$\\,/\\,$s)")
plt.show()

# Simulation 2D

In [ ]:
# plot max-e-field energy
# TODO: Change markers & colors, perhaps change the format completely so that there is more space for the different markers (in that case focus on 100 and 140 km/s)
plt.style.use(MPLSTYLE_FILE)
plt.figure()
for files, label, marker in zip([
        sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")),
        sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx").glob("*.h5")),
        sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")),
        sorted((FOLDER_2D / "v_alpha_bulk_variation_Bz").glob("*.h5")),
        sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")),
    ], ["B=0", "B_x>0", "B_y>0", "B_z>0", "B_x=B_y"], settings.MARKERS):
    velocity = np.empty(len(files))
    W_E_max = np.empty(len(files))
    W_E_max_err = np.empty(len(files))
    for file_idx, filename in enumerate(files):
        velocity[file_idx] = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            E_x = f['Electric Field/ex'][:]
            E_y = f['Electric Field/ey'][:]
        W_E = np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)) * (constants.epsilon_0 / 2) / constants.electron_volt
        max_idx = np.argmax(W_E)
        max_range = W_E[max_idx-5:max_idx+5]
        W_E_max[file_idx] = np.mean(max_range)
        W_E_max_err[file_idx] = np.std(max_range)
    velocity = np.array(velocity)
    W_E_max = np.array(W_E_max)
    W_E_max_err = np.array(W_E_max_err)
    if True:
        K_alpha_t0 = (info.alpha.si_mass * info.alpha.number_density * (velocity * 1e3) ** 2 / (2 * constants.electron_volt))
        W_E_max /= K_alpha_t0
        W_E_max_err /= K_alpha_t0

    plt.errorbar(
        velocity, W_E_max, yerr=W_E_max_err, label=f"${label}$", # color="black",
        marker=marker, markeredgecolor="black", markeredgewidth=1, markersize=10, ls="")
    y_min, y_max = plt.gca().get_ylim()
plt.fill_between(
    [0.0, info.c_s * 1e-3], y1=y_min, y2=y_max,
    lw=2, color="lightgray", label="$u_{\\alpha} \\leq c_s$"
)
plt.xlabel("Initial flow velocity $u_{\\alpha}^{t=0}$ (km/s)")
if True:
    plt.ylabel("$\\max[\\langle W_E\\rangle_\\mathbf{r}]_t/K_\\alpha^{t=0}$  (1)")
else:
    plt.ylabel("$\\max[\\langle W_E\\rangle_\\mathbf{r}]_t$  (MeV$\\,/\\,$m$^3$)")
plt.xlim(90, 190)
plt.ylim(y_min, y_max)
plt.legend(fancybox=False, edgecolor="black")
plt.tight_layout()
# plt.savefig("figures/svg/simulation-2D/magnetic-field-directions/max_electric_field_energy-vs-flow-velocity.svg", bbox_inches="tight")
plt.show()

## Alpha flow-speed variation (B=0)

In [ ]:
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=False)
plt.show()
plots_2D.maxEnergyVsAlphaFlowSpeed(info, normalize_energy=True, save=save)
plt.show()

In [ ]:
plots_2D.waveAngleVsAlphaFlowSpeed(info, "x", save=save)
plots_2D.simulatedAlphaFlowSpeed(info, "x", save=save)
plt.show()

TODO: |k| vs u-alpha

In [ ]:
plots_2D.omegaVsAlphaFlowSpeed(info, "x", save=save)
plots_2D.omegaVsAlphaFlowSpeed(info, "y", save=save)
plt.show()

In [ ]:
plots_2D.psdOmegaForAlphaFlowSpeed(info, "x", save=save)
plots_2D.psdOmegaForAlphaFlowSpeed(info, "y", save=save)
plt.show()

In [ ]:
import theory
dr = np.logspace(-3, 1)
plt.plot(dr, theory.growthRate(dr), label="$\\gamma\\,/\\,\\omega_{pp}$")
plt.plot(dr, theory.waveFrequency(dr), label="$\\omega\\,/\\,\\omega_{pp}$")
plt.plot(dr, theory.waveNumber(dr, info), label="$k\\,\\lambda_D$")
plt.xscale("log")
plt.xlim(1e-3, 1e1)
plt.xlabel("Density ratio $n_\\alpha\\,/\\,n_{p}$ (1)")
plt.ylabel("Ratio (1)")
plt.legend()
plt.show()

In [ ]:
# plot 3D temperature vs time for different magnetic fields.
species = Species.PROTON
plt.style.use(MPLSTYLE_FILE)
plt.figure()

leg_u100 = []
leg_u140 = []
for u_label in [100, 140]:
    for folder_id, label in zip(["", "_Bx", "_By", "_Bz", "_Bx_By"], ["B=0", "B_x>0", "B_y>0", "B_z>0", "B_x=B_y"]):
        for filename in sorted((FOLDER_2D / f"v_alpha_bulk_variation{folder_id}").glob("*.h5")):
            u = int(filename.stem[-3:])
            if u != u_label:
                continue
            with h5py.File(filename) as f:
                time = f["Header/time"][:] * info.omega_pp
                temp = np.mean(f[f"Derived/Temperature/{species.value}"], axis=(1,2))
            line = plt.plot(
                time, physics.kelvinToElectronVolt(temp),
                # label=f"(${label}$) $u_\\alpha^{{t=0}}$ = {v}"
                label=f"${label}$"
            )[0]
            (leg_u100 if u_label == 100 else leg_u140).append(line)
leg1 = plt.legend(handles=leg_u100, loc="upper left", title="$u_\\alpha^{t=0}$=100 km/s",ncols=1 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.legend(handles=leg_u140, title="$u_\\alpha^{t=0}$=140 km/s", loc="lower right", ncols=3 if species == Species.ELECTRON else 2, columnspacing=0.5, labelspacing=0.5, fancybox=False, framealpha=0.4)
plt.gca().add_artist(leg1)
plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
plt.ylabel(f"Temperature $T_{species.symbol()}$ (eV)")
plt.xlim(0, 150.0)
y_low = 1.5 if species == Species.PROTON else 5
plt.ylim(bottom=2 if species == Species.PROTON else 10 if species == Species.ALPHA else 99.7)
plt.tight_layout(pad=0.2)
plt.savefig(f"figures/svg/simulation-2D/magnetic-field-directions/temperature_{species}-vs-time.svg", bbox_inches="tight")
plt.show()

In [ ]:
for species in Species:
    plots_2D.temperature3DOverTimeForAlphaFlowSpeed(
        info, species
    )

In [ ]:
for species in Species:
    plots_2D.temperatureDifferences3DVsAlphaFlowSpeed(info, species, save=save)
plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_By").glob("*.h5")):
    plots_2D.electricField2DSnapshot(filename, info, time=50.0)
    plt.show()

In [ ]:
files = sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5"))
arr_theta = np.empty(len(files))
arr_theta_err = np.empty(len(files))
arr_theta_E = np.empty(len(files))
arr_theta_E_err = np.empty(len(files))
flow_velocity = np.empty(len(files))
for file_idx, filename in enumerate(files):
    flow_velocity[file_idx] = int(filename.stem[-3:])
    with h5py.File(filename) as f:
        x = f["Grid/x_px/Alphas/X"][0] / info.lambda_D
        y = f["Grid/y_px/Alphas/Y"][0] / info.lambda_D
        E_x = f['Electric Field/ex'][:]
        E_y = f['Electric Field/ey'][:]
        time = f["Header/time"][:] * info.omega_pp

    res = analysis.fitGrowthRate(time, np.mean(E_x ** 2 + E_y ** 2, axis=(1,2)))
    assert res is not None, "What?"
    linear_idx = res[1]
    E_field = E_x[slice(*linear_idx)]
    k, k_err = analysis.waveVector2D(x, y, E_field)
    print(k, k_err)
    print(f"v = {flow_velocity[file_idx]} :: |k| = {np.linalg.norm(k):.4f}")
    arr_theta[file_idx], arr_theta_err[file_idx] = analysis.waveAngle2DFromWaveVector(k, k_err)

print(arr_theta * 180 / np.pi)

In [ ]:
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx_By").glob("*.h5")):
    plots_2D.energyEFieldOverTime(filename, info)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.strengthBFieldOverTime(filename, info)
    plots_2D.psdBField(filename, info)
    plots_2D.energyBField(filename, info)
    plt.show()

In [ ]:
for species in [Species.ELECTRON]:
    plots_2D.psdFlowVelocity(info, species, "x", "x")
    plt.show()

In [ ]:
for species in Species:
    plots_2D.flowVelocityVsTime(info, species, "x", "x")
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plots_2D.energiesOverTime(filename, info)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    K_species = []
    for species in Species:
        u_alpha = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/grid/X"][:]
            y_grid = f[f"Grid/grid/Y"][:]
            px_grid = f[f"Grid/px_py/{species.value}/Px"][:]
            py_grid = f[f"Grid/px_py/{species.value}/Py"][:]
            px_py = f[f'dist_fn/px_py/{species.value}'][:]
            print(f[f"Derived/Temperature/{species}"].keys())
        # TODO: Wrong
        u_x, u_y = analysis.flowVelocity2D(x_grid, y_grid, px_grid, py_grid, px_py, info[species])
        K = 0.5 * info[species].si_mass * (u_x ** 2 + u_y ** 2) / constants.electron_volt


In [ ]:
# Kinetic energies in the system
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    plt.figure()
    K_species = []
    for species in Species:
        u_alpha = int(filename.stem[-3:])
        with h5py.File(filename) as f:
            time = f["Header/time"][:] * info.omega_pp
            x_grid = f[f"Grid/grid/X"][:]
            y_grid = f[f"Grid/grid/Y"][:]
            px_grid = f[f"Grid/px_py/{species.value}/Px"][:]
            py_grid = f[f"Grid/px_py/{species.value}/Py"][:]
            px_py = f[f'dist_fn/px_py/{species.value}'][:]
        # TODO: Wrong
        u_x, u_y = analysis.flowVelocity2D(x_grid, y_grid, px_grid, py_grid, px_py, info[species])
        K = 0.5 * info[species].si_mass * (u_x ** 2 + u_y ** 2) / constants.electron_volt
        K_species.append(info[species].number_density * K)
    for K_s, species in zip(K_species, Species):
        plt.plot(time, K_s * 1e-6, label=f"$W_{{{species.symbol()}}}$")
    # plt.plot(time[1:], W_total[1:] * 1e-6, label="$W_\\text{total}$")
    plt.yscale("log")
    plt.xlabel("Time $t\\,\\omega_{pp}$ (1)")
    plt.ylabel("Kinetic energy density K (MeV$\\,/\\,$m$^3$)")
    plt.xlim(0.0, 150.0)
    plt.legend(ncols=3, labelspacing=.2, columnspacing=0.5)
    plt.show()
    break

In [ ]:
species = Species.PROTON
files = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
labels = [int(p.stem[-3:]) for p in files]
general.momentumDistributionComparison(
    info, species, Distribution.X_PX, legend_ncols=2,
    files=files, labels=labels, times=150.0, legend_title=f"Flow velocity $u_\\alpha$ (km/s)")
plt.show()

In [ ]:
# TODO: Test with (FOLDER_2D / "v_alpha_bulk_variation_Bx_By") (varying momentum limits)
times = [0.0, 40.0, 50, 60.0, 150.0]
species = Species.PROTON
for filename in sorted((FOLDER_2D / "v_alpha_bulk_variation_Bx").glob("*.h5")):
    general.momentumDistributionComparison(info, species, Distribution.Y_PX, filename, times)
    break
plt.show()

In [ ]:
species = Species.ALPHA
dist = Distribution.X_PX
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    general.spaceMomentumDistributon(info, species, dist, filename, time=50.0)
    plt.show()

In [ ]:
species = Species.ALPHA
time = 60.0
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    print(filename.stem[-3:])
    plots_2D.pxPyDistribution(info, species, filename, time)
    plt.show()

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    label = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoEFieldOverTime(info, filename, "x", label=label, save=True)

In [ ]:
filenames = sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5"))
species = Species.PROTON
normalized_velocity = True
time_steps = range(0, 1500, 20)
labels = [int(p.stem[-3:]) for p in filenames]
legend_title=f"Flow velocity $u_\\alpha$ (km/s)"
general.videoMomentumDistribution(
    info, Distribution.X_PX, species, filenames,
    save=True, labels=labels, legend_title=legend_title,
    legend_ncols=2)

In [ ]:
for filename in sorted(V_FLOW_VARIATION_FOLDER.glob("*.h5")):
    if filename.stem[-3:] not in ["100", "140", "180"]:
        print("Skip")
        continue
    v = f"u_alpha={filename.stem[-3:]}"
    plots_2D.videoPxPyDistribution(info, species=Species.PROTON, filename=filename, save=True, label=v)

In [ ]:
filename = RESULTS_FOLDER / "epoch_2D/v_alpha_bulk_variation/v_alpha_bulk_106.h5"
v_x, v_y, f_v = general._loadPxPyDistribution(info, Species.PROTON, filename, 100, True)
f_v[f_v<=0] = np.min(f_v[f_v>0])
from scipy import ndimage
plt.contourf(np.log(ndimage.gaussian_filter(f_v, sigma=10)))
plt.gca().set_aspect("equal")

# Simulation 1D

## Density variation

In [ ]:
for filename in sorted(DENSITY_VARIATION_FOLDER.glob("density_*.h5")):
    print(filename)
    ratio = 10 ** float(filename.stem.split("_")[-1])
    n_electron = 12e6
    n_proton = n_electron / (1 + 2 * ratio)
    n_alpha = ratio * n_proton
    info = RunInfo(
        electron=SpeciesInfo(
            number_density=12.0e6,
            temperature=100.0,
            charge=-1,
            mass=1.0,
            bulk_velocity=0.0
        ),
        proton=SpeciesInfo(
            number_density=n_proton,
            temperature=3.0,
            charge=+1,
            mass=1836.152674,
            bulk_velocity=0.0
        ),
        alpha=SpeciesInfo(
            number_density=n_alpha,
            temperature=12.0,
            charge=+2,
            mass=7294.29953,
            bulk_velocity=1.0e5
        )
    )
    plots_1D.energyEFieldOverTime(
        filename=filename, info=info
    )
    plt.show()

## Run n=8192

In [ ]:
plots_1D.electricFieldOverSpaceAndTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=False
)
plt.show()

In [ ]:
plots_1D.averageTemperature3DOverTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

In [ ]:
for s in Species:
    plots_1D.velocityDistributionOverTime(
        filename=PARTICLE_VARIATION_FOLDER / "particles_4192/rep_0.h5", #FOLDER_1D / "proton-alpha-instability-1D.h5",
        info=info, species=s, save=save
    )
plt.show()

In [ ]:
plots_1D.energyEFieldOverTime(
    filename=FOLDER_1D / "proton-alpha-instability-1D.h5",
    info=info, save=save
)
plt.show()

## Particle variation

In [ ]:
variation_folder = PARTICLE_VARIATION_FOLDER
for particle_folder in sorted(variation_folder.glob("particles_*")):
    plots_1D.multiElectricFieldEnergyOverTime(
        particle_folder, info, save=save,
        identifer=f"particles_per_cell_{int(particle_folder.stem[-4:])}"
    )

In [ ]:
plots_1D.particleVariationEnergyVsTime(info=info, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperature3D(species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXDiff(info, species, save=save)
plt.show()

In [ ]:
for species in Species:
    plots_1D.particleVariationTemperatureXVsTime(info, species, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationGrowthRate(info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationWavenumber(info=info, save=save)
plt.show()

In [ ]:
plots_1D.particleVariationFrequency(info=info, save=save)
plt.show()